# Distributional Model(s)

### Gaussian Process Regressor

In [ ]:
def get_GPR(X_train_in,X_test_in,y_means_in):
    # Initialize Cross-Vlidator of GPR
    CV_GPR = RandomizedSearchCV(estimator=GaussianProcessRegressor(),
                                n_jobs=n_jobs,
                                cv=KFold(2, random_state=2020, shuffle=True),
                                param_distributions=param_grid_GAUSSIAN,
                                n_iter=n_iter,
                                return_train_score=True,
                                random_state=2021,
                                verbose=10)

    CV_GPR.fit(X_train,Y_train_mean_emp)
    # Get Best Model
    best_GPR = CV_GPR.best_estimator_

    # Get Training-Set Prediction
    GPR_means = best_GPR.predict(X_train,return_std=True)[0]
    GPR_vars = (best_GPR.predict(X_train,return_std=True)[1])**2

    # Get Test-Set Predictions
    GPR_means_test = best_GPR.predict(X_test,return_std=True)[0]
    GPR_vars_test = (best_GPR.predict(X_test,return_std=True)[1])**2
    
    # Return Trained Predictions + Model
    GPR_means_test = best_GPR.predict(X_test,return_std=True)[0]
    return GPR_means,GPR_vars, GPR_means_test, GPR_vars_test, best_GPR

# Universal Gaussian DNN

Maps $\varrho:\mathbb{R}^d\ni \to (\hat{\mu},\sigma)\in \mathbb{R}\times (0,\infty)$.  

Implictly:
$
\rho:\mathbb{R}^d\ni \to \nu\circ \varrho(x)\in \mathcal{P}_2(\mathbb{R})
.
$

The universal approximation theorem for this architecture is given in [Corollary 7: Quantitative Rates and Fundamental Obstructions to Non-EuclideanUniversal Approximation with Deep Narrow Feed-Forward Networks](https://arxiv.org/pdf/2101.05390.pdf)

In [2]:
class Gaussian_Splitter(tf.keras.layers.Layer):

    def __init__(self):
        super(fullyConnected_Dense, self).__init__()
        self.units = units

    def call(self):
        return tf.math.pow(self,2)

NameError: name 'tf' is not defined

In [4]:
def get_ffNN_Gaussian(height, depth, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
   
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.swish(core_layers)
    # Train additional Depth?
    if depth>1:
        # Add additional deep layer(s)
        for depth_i in range(1,depth):
            core_layers = fullyConnected_Dense(height)(core_layers)
            # Activation
            core_layers = tf.nn.swish(core_layers)
    core_layers = fullyConnected_Dense(output_dim)(core_layers)  
    #------------------#
    #  Readout Layers  #
    #------------------# 
    # Gaussian Splitter Layer
    output_layers = Gaussian_Splitter(core_layers)
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model



def build_ffNN_Gaussian(n_folds , n_jobs, n_iter, param_grid_in, X_train, y_train,X_test):
    # Update Dictionary
    param_grid_in_internal = param_grid_in
    param_grid_in_internal['input_dim'] = [(X_train.shape[1])]
    
    # Deep Feature Network
    ffNN_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=get_ffNN_Gaussian, 
                                                            verbose=True)
    
    # Randomized CV
    ffNN_CVer = RandomizedSearchCV(estimator=ffNN_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(n_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_in_internal,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit Model #
    #-----------#
    ffNN_CVer.fit(X_train,y_train)

    # Write Predictions #
    #-------------------#
    y_hat_train = ffNN_CVer.predict(X_train)
    
    eval_time_ffNN = time.time()
    y_hat_test = ffNN_CVer.predict(X_test)
    eval_time_ffNN = time.time() - eval_time_ffNN
    
    # Counter number of parameters #
    #------------------------------#
    # Extract Best Model
    best_model = ffNN_CVer.best_estimator_
    # Count Number of Parameters
    N_params_best_ffNN = np.sum([np.prod(v.get_shape().as_list()) for v in best_model.model.trainable_variables])
    
    
    # Return Values #
    #---------------#
    return y_hat_train, y_hat_test, N_params_best_ffNN, eval_time_ffNN

# Update User
#-------------#
print('Deep Feature Builder - Ready')

Deep Feature Builder - Ready


---

#### Gaussian Output Layer
UAP-Preserving
$\rho:\mathbb{R}^d\ni x \mapsto (x_1,\exp(x_2))\in \mathbb{R}\times (0,\infty)$; Thus, $\rho_{\star}[\mathcal{NN}_{d,2}^{\sigma}]$ is dense in $C(\mathbb{R}^d,\mathbb{R}\times [0,\infty))$ by [this paper's main result.](https://proceedings.neurips.cc/paper/2020/hash/786ab8c4d7ee758f80d57e65582e609d-Abstract.html).

In [ ]:
# Affine Readout post-composed with UAP-preserving readout map to G_d
class Gaussian_Splitter(tf.keras.layers.Layer):

    def __init__(self, units=16, input_dim=32):
        super(Gaussian_Splitter, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(name='Weights_ffNN',
                                 shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
        self.b = self.add_weight(name='bias_ffNN',
                                 shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

    def call(self, inputs):
        parameters = tf.matmul(inputs, self.w) + self.b
        mean_and_cov = tf.concat([parameters,tf.math.exp(parameters)],-1)
        return mean_and_cov

Implements the above deep network.

In [ ]:
def get_ffNN_Gaussian(height, depth, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
   
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.swish(core_layers)
    # Train additional Depth?
    if depth>1:
        # Add additional deep layer(s)
        for depth_i in range(1,depth):
            core_layers = fullyConnected_Dense(height)(core_layers)
            # Activation
            core_layers = tf.nn.swish(core_layers)
    
    #------------------#
    #  Readout Layers  #
    #------------------# 
    # Gaussian Splitter Layer
    output_layers = Gaussian_Splitter(output_dim)(core_layers)  
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model



def build_ffNN_Gaussian(n_folds , n_jobs, n_iter, param_grid_in, X_train, y_train,X_test):
    # Update Dictionary
    param_grid_in_internal = param_grid_in
    param_grid_in_internal['input_dim'] = [(X_train.shape[1])]
    
    # Deep Feature Network
    ffNN_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=get_ffNN_Gaussian, 
                                                            verbose=True)
    
    # Randomized CV
    ffNN_CVer = RandomizedSearchCV(estimator=ffNN_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(n_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_in_internal,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit Model #
    #-----------#
    ffNN_CVer.fit(X_train,y_train)

    # Write Predictions #
    #-------------------#
    y_hat_train = ffNN_CVer.predict(X_train)
    
    eval_time_ffNN = time.time()
    y_hat_test = ffNN_CVer.predict(X_test)
    eval_time_ffNN = time.time() - eval_time_ffNN
    
    # Counter number of parameters #
    #------------------------------#
    # Extract Best Model
    best_model = ffNN_CVer.best_estimator_
    # Count Number of Parameters
    N_params_best_ffNN = np.sum([np.prod(v.get_shape().as_list()) for v in best_model.model.trainable_variables])
    
    
    # Return Values #
    #---------------#
    return y_hat_train, y_hat_test, N_params_best_ffNN, eval_time_ffNN

# Update User
#-------------#
print('Deep Feature Builder - Ready')

---

# Run Model(s)

### Perform Gaussian Process Regression

In [ ]:
GPR_means, GPR_vars, GPR_means_test, GPR_vars_test, trash = get_GPR(X_train,X_test,Y_train_mean_emp) 

## Deep Gaussian Networks

In [ ]:
print("===============================")
print("Training Deep Gaussian Network!")
print("===============================")

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [problem_dim]
param_grid_Deep_Classifier['output_dim'] = [1]

# Train simple deep classifier
Deep_Gaussian_train_parameters, Deep_Gaussian_test_parameters, N_params_deep_Gaussian, timer_output_Deep_Gaussian = build_ffNN_Gaussian(n_folds = CV_folds, 
                                                                                                                                        n_jobs = n_jobs, 
                                                                                                                                        n_iter = n_iter, 
                                                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                                                        X_train = X_train, 
                                                                                                                                        y_train = Y_train_var_emp,
                                                                                                                                        X_test = X_test)

print("====================================")
print("Training Deep Gaussian Network!: END")
print("====================================")